# Это линейная регрессия с регуляризацией

Мы уже успели реализовать линейную регрессию на лекции 

 $Linear\quad Regression = \langle x , w \rangle $

Оптимизировали же мы нашу модель, уменьшая евклидово расстояние между примерами и предсказаниями нашей модели : $MSE= \sum_i(y_i-x_i^Tw)^2=||Y-Xw||_2^2$
Однако так модель может найти "идеальные" или около того параметры на тренировочных данных, однако тренировочные данные не позволяют модели обобщиться, а значит не позволяют добиться того, чего мы хочем.
Чтобы модель не "подгонялась" под тренировочные данные люди решили добавлять в функцию потерь штраф за большие веса(статистически доказано, что переобучение связано с высокими весами). Методы, назначенные на борьбу с переобучением моделей называются регуляризацией.
Собственно говоря, здесь я реализовал один из способов - ridge-regression.

In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")